<a href="https://colab.research.google.com/github/sunbooshi/1Click-Bert-VITS2/blob/main/1ClickBertVITS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 环境准备



In [ ]:
#@title 安装Bert-VITS2
!git clone https://github.com/fishaudio/Bert-VITS2
%cd Bert-VITS2
!git checkout v2.3
%pip install -r requirements.txt
%cd /content/

In [ ]:
#@title 安装audio-slicer
!git clone https://github.com/openvpi/audio-slicer.git audio_slicer

In [ ]:
#@title 安装whisper
!pip install git+https://github.com/openai/whisper.git
!pip install torchaudio

In [ ]:
#@title 安装ffmpeg
import os, uuid, re, IPython
import ipywidgets as widgets
import time

from glob import glob
from google.colab import output, drive

from IPython.display import clear_output
import os, sys, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/yunooooo/gcct/master/res/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Cloning Repositories...", ty='twg')
!git clone https://github.com/XniceCraft/ffmpeg-colab.git
!chmod 755 ./ffmpeg-colab/install
textAn("Installing FFmpeg...", ty='twg')
!./ffmpeg-colab/install
clear_output()
print('Installation finished!')
!rm -fr /content/ffmpeg-colab
!ffmpeg -version

In [ ]:
#@title 准备目录
%cd /content/Bert-VITS2/
import os
data_dir = 'Data'
os.mkdir(data_dir)
speaker = "paimeng" # @param {type:"string"}
speaker_dir = os.path.join(data_dir, speaker)
os.mkdir(speaker_dir)
audios = os.path.join(speaker_dir, 'audios')
os.mkdir(audios)
audios_raw = os.path.join(audios, 'raw')
os.mkdir(audios_raw)
audios_wavs = os.path.join(audios, 'wavs')
os.mkdir(audios_wavs)
filelists = os.path.join(speaker_dir, 'filelists')
os.mkdir(filelists)
input_dir = "inputs"
os.mkdir(input_dir)
models_dir = os.path.join(speaker_dir, 'models')
os.mkdir(models_dir)

# 模型下载

In [ ]:
#@title 下载模型
!wget -P slm/wavlm-base-plus/ https://huggingface.co/microsoft/wavlm-base-plus/resolve/main/pytorch_model.bin
!wget -P emotional/clap-htsat-fused/ https://huggingface.co/laion/clap-htsat-fused/resolve/main/pytorch_model.bin
!wget -P emotional/wav2vec2-large-robust-12-ft-emotion-msp-dim/ https://huggingface.co/audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim/resolve/main/pytorch_model.bin
!wget -P bert/chinese-roberta-wwm-ext-large/ https://huggingface.co/hfl/chinese-roberta-wwm-ext-large/resolve/main/pytorch_model.bin
!wget -P bert/bert-base-japanese-v3/ https://huggingface.co/cl-tohoku/bert-base-japanese-v3/resolve/main/pytorch_model.bin
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/pytorch_model.bin
!wget -P bert/deberta-v3-large/ https://huggingface.co/microsoft/deberta-v3-large/resolve/main/pytorch_model.generator.bin
!wget -P bert/deberta-v2-large-japanese/ https://huggingface.co/ku-nlp/deberta-v2-large-japanese/resolve/main/pytorch_model.bin

In [ ]:
# @title 训练底模
!wget -P $models_dir https://huggingface.co/OedoSoldier/Bert-VITS2-2.3/resolve/main/DUR_0.pth
!wget -P $models_dir https://huggingface.co/OedoSoldier/Bert-VITS2-2.3/resolve/main/D_0.pth
!wget -P $models_dir https://huggingface.co/OedoSoldier/Bert-VITS2-2.3/resolve/main/G_0.pth
!wget -P $models_dir https://huggingface.co/OedoSoldier/Bert-VITS2-2.3/resolve/main/WD_0.pth

# 准备训练数据

## 上传训练数据

In [ ]:
# @title ### 直接上传
import shutil
from google.colab import files

uploaded = files.upload()

for f in uploaded.keys():
  shutil.move(f, input_dir)

### 从GoogleDrive 复制

In [ ]:
# @title #### 挂载GoogleDrive
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
# @title #### 复制文件
import shutil
# @markdown  输入要复制的文件路径：
drive_path = "/content/drive/MyDrive/data/paimeng.wav" # @param {type:"string"}
if os.path.isdir(drive_path):
  !cp {drive_path}*.wav {input_dir}
else:
  !cp {drive_path} {input_dir}

## 数据处理

In [ ]:
# @title ### 音频分割
import librosa  # Optional. Use any library you like to read audio files.
import soundfile  # Optional. Use any library you like to write audio files.
import os
import sys
sys.path.append('/content/audio_slicer')
from slicer2 import Slicer

def slice_audio(in_file, out_path, out_name, base_index=0):
    audio, sr = librosa.load(in_file, sr=None, mono=False)  # Load an audio file with librosa.
    slicer = Slicer(
        sr=sr,
        threshold=-40,
        min_length=2000,
        min_interval=300,
        hop_size=10,
        max_sil_kept=500
    )
    chunks = slicer.slice(audio)
    total = 0
    for i, chunk in enumerate(chunks):
        total = total + 1
        if len(chunk.shape) > 1:
            chunk = chunk.T  # Swap axes if the audio is stereo.
        out_file = os.path.join(out_path, '%s_%03d.wav' % (out_name, base_index+i))
        soundfile.write(out_file, chunk, sr)  # Save sliced audio files with soundfile.

    return total

def slice_dir(input_dir, output_dir, model_name):
    # 列出当前工作目录中的所有文件和文件夹
    files = os.listdir(input_dir)

    base_index = 0
    # 筛选出所有以 ".wav" 结尾的文件
    wav_files = [f for f in files if f.endswith(".wav")]

    # 打印所有 wav 文件的名称
    for f in wav_files:
        base_index = base_index + slice_audio(os.path.join(input_dir, f), output_dir, model_name, base_index)


if len(input_dir)>0 and len(audios_raw)>0 and len(speaker)>0:
  slice_dir(input_dir, audios_raw, speaker)
else:
  print("error input")

print("分割完成")

In [ ]:
# @title ### 音频标注

import whisper
import os
import json
import torchaudio
import argparse
import torch

lang2token = {
            'zh': "ZH|",
            'ja': "JP|",
            "en": "EN|",
        }

def transcribe(audio_path):
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")
    lang = max(probs, key=probs.get)
    # decode the audio
    options = whisper.DecodingOptions(beam_size=5)
    result = whisper.decode(model, mel, options)

    # print the recognized text
    print(result.text)
    return lang, result.text


# assert (torch.cuda.is_available()), "Please enable GPU in order to run Whisper!"
model = whisper.load_model("medium")
speaker_annos = []
total_files = sum([len(files) for r, d, files in os.walk(audios_raw)])
processed_files = 0

for i, wavfile in enumerate(list(os.walk(audios_raw))[0][2]):
    # try to load file as audio
    try:
        lang, text = transcribe(os.path.join(audios_raw, wavfile))
        if lang not in list(lang2token.keys()):
            print(f"{lang} not supported, ignoring\n")
            continue

        text = f"{audios_wavs}/{wavfile}|" + f"{speaker}|" +lang2token[lang] + text + "\n"
        speaker_annos.append(text)

        processed_files += 1
        print(f"Processed: {processed_files}/{total_files}")
    except Exception as e:
        print(e)
        continue

if len(speaker_annos) == 0:
    print("Warning: no short audios found, this IS expected if you have only uploaded long audios, videos or video links.")
    print("this IS NOT expected if you have uploaded a zip file of short audios. Please check your file structure or make sure your audio language is supported.")
with open(os.path.join(filelists, 'esd.list'), 'w', encoding='utf-8') as f:
    for line in speaker_annos:
        f.write(line)

print("标注完成")

In [ ]:
# @title ### 配置文件生成
log_interval = 50 # @param {type:"integer"}
eval_interval = 100 # @param {type:"integer"}
epochs = 100 # @param {type:"integer"}
batch_size = 4 # @param {type:"integer"}
bert_device = "cuda" # @param ["cuda", "cpu"]
bert_processs = 2 # @param {type:"integer"}

import json
with open('configs/config.json') as fp:
  configs = json.load(fp)

configs['train']['log_interval'] = log_interval
configs['train']['eval_interval'] = eval_interval
configs['train']['epochs'] = epochs
configs['train']['batch_size'] = batch_size

configs['data']['training_files'] = os.path.join(filelists, 'train.list')
configs['data']['validation_files'] = os.path.join(filelists, 'val.list')
configs['data']['n_speakers'] = 1
configs['data']['spk2id'] = {speaker : 0}

configs_path = os.path.join(speaker_dir, 'config.json')
with open(configs_path, 'w') as fp:
  json.dump(configs, fp, indent=4)
  print(f'生成配置文件: {configs_path}')

configs = None
import yaml
with open('default_config.yml') as fp:
  configs = yaml.load(fp, Loader=yaml.FullLoader)

configs['dataset_path'] = speaker_dir
configs['preprocess_text']['transcription_path'] = 'filelists/esd.list'
configs['bert_gen']['device'] = bert_device
configs['bert_gen']['num_processes'] = bert_processs

with open('config.yml', 'w') as fp:
  yaml.dump(configs, fp)
  print(f'生成配置文件: config.yml')

In [ ]:
#@title 音频重采样
!python resample.py

In [ ]:
#@title 生成训练集、验证集
!python preprocess_text.py

In [ ]:
#@title 生成Bert
!python bert_gen.py

# Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $models_dir

# 训练

In [ ]:
!python train_ms.py

# 推理

In [ ]:
model = "G_1000.pth" # @param {type:"string"}

configs = None
import yaml
with open('config.yml') as fp:
  configs = yaml.load(fp, Loader=yaml.FullLoader)
  configs['webui']['model'] = f'models/{model}'
  configs['webui']['share'] = True

with open('config.yml', 'w') as fp:
  yaml.dump(configs, fp)
  print(f'更新配置文件: config.yml')

print('启动webui')
!python webui.py